# Example script to regrid CESM2 sea ice concentration to a regular 1$^\circ$x1$^\circ$ grid using xesmf

Start by loading appropriate packages

In [1]:
import xarray as xr
import numpy as np
import xesmf as xe

Get historical sea ice concentration data from CESM2 for member r1i1p1f1

In [3]:
direc_i = '/Users/pade7652/Google Drive/CUBoulder/Projects/CESM2/Data/CESM2-CMIP6/' # Change to match the directory where your data is found
xdi = xr.open_dataset(direc_i+'siconc_SImon_CESM2_historical_r1i1p1f1_gn_185001-201412.nc',chunks={}) # The 'chunks' option above tells xarray to make this a dask array
print(xdi)

<xarray.Dataset>
Dimensions:    (d2: 2, ni: 320, nj: 384, nvertices: 4, time: 1980)
Coordinates:
    lat        (nj, ni) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    lon        (nj, ni) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * ni         (ni) int32 1 2 3 4 5 6 7 8 9 ... 313 314 315 316 317 318 319 320
  * nj         (nj) int32 1 2 3 4 5 6 7 8 9 ... 377 378 379 380 381 382 383 384
  * time       (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
Dimensions without coordinates: d2, nvertices
Data variables:
    siconc     (time, nj, ni) float32 dask.array<chunksize=(1980, 384, 320), meta=np.ndarray>
    time_bnds  (time, d2) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat_bnds   (nj, ni, nvertices) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
    lon_bnds   (nj, ni, nvertices) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity

If the coordinate variables are named something different than 'lat' and 'lon', rename them to 'lat' and 'lon', otherwise xesmf has no way of knowing what these are (it can't guess what your model's lat/lon coords are named).
It can only recognize coordinates named 'lon' and 'lat'.

In [5]:
# xdi = xdi.rename({'latitude':'lat','longitude':'lon'})

Select the 'siconc' variable. Can also select specific years and/or months at this point.

In [9]:
sic = xdi['siconc']
# sic = xdi['siconc'].sel({'time':slice('1980-01-01','2000-01-01')})
print(sic)

<xarray.DataArray 'siconc' (time: 1980, nj: 384, ni: 320)>
dask.array<open_dataset-49a09dc33c4112a84ad64670f8eb4236siconc, shape=(1980, 384, 320), dtype=float32, chunksize=(1980, 384, 320), chunktype=numpy.ndarray>
Coordinates:
    lat      (nj, ni) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    lon      (nj, ni) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * ni       (ni) int32 1 2 3 4 5 6 7 8 9 ... 313 314 315 316 317 318 319 320
  * nj       (nj) int32 1 2 3 4 5 6 7 8 9 ... 377 378 379 380 381 382 383 384
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
Attributes: (12/19)
    cell_measures:  area: areacello
    cell_methods:   area: mean where sea time: mean
    comment:        Area fraction of grid cell covered by sea ice
    description:    Area fraction of grid cell covered by sea ice
    frequency:      mon
    id:             siconc
    ...             ...
    time_label:     time-mean
    time_title:     Temporal mean
    ti

Define the target grid (the new grid you want your output on).

In [15]:
# In this case, we want to regrid to a 1 deg lat x 1 deg lon grid
lat = np.arange(-89.0,90.0,1.0) # Do not include 90S and 90N, it won't work because of the singularity at the poles
lon = np.arange(0.0,361.0,1.0) # cartopy gets rage-y (i.e. it doesn't work) when plotting
lon2d,lat2d = np.meshgrid(lon,lat)
target_grid = xr.Dataset({'lat': (['y', 'x'], lat2d),'lon': (['y', 'x'], lon2d)})

Set up the regridder:

In [16]:
# Possible regridding algorithms. For a comparison of the different algorithms, check out: https://xesmf.readthedocs.io/en/latest/notebooks/Compare_algorithms.html
method_list = ['bilinear', 'conservative', 'nearest_s2d', 'nearest_d2s', 'patch']
# NOTE: Once you have the output of this (the weight file), you can set reuse_weights=True so that you don't need to recreate the weight file
regridder = xe.Regridder(sic, target_grid, method_list[2], periodic=True, reuse_weights=True)

# Regrid
sic_rg = regridder(sic)

ValueError: To reuse weights, you need to provide either filename or weights.